In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm

In [ ]:
from pathlib import Path
import pandas as pd

# Base directory (3 levels up from current notebook if needed)
base_dir = Path(__file__).resolve().parents[1]  # adjust depending on location

# Paths
input_excel = base_dir / "20211030 Test Assignment" / "Input.xlsx"
output_dir = base_dir / "20211030 Test Assignment" / "Aritcle_txt_files"

# Make sure output directory exists
output_dir.mkdir(parents=True, exist_ok=True)

# Load input file
df = pd.read_excel(input_excel)


FileNotFoundError: [Errno 2] No such file or directory: 'BlackCoffer-TestAssginment\\20211030 Test Assignment\\Input.xlsx'

In [3]:
len(input['URL'])

147

In [ ]:
from tqdm import tqdm
import requests
import time

t_url = {}            # holds successful responses
urls = list(input['URL'])

# Keep looping until we've fetched every URL
while True:
    # Find which URLs are still missing
    missing = [url for url in urls if url not in t_url]
    if not missing:
        break  # done!

    # Retry each missing URL once per iteration
    for url in tqdm(missing, desc="Retrying missing URLs"):
        try:
            resp = requests.get(url, timeout=10)
            if resp.status_code == 200:
                t_url[url] = resp
        except requests.RequestException as e:
            # swallow the exception and move on; we'll retry in the next pass
            print(f"Error fetching {url}: {e}")
        time.sleep(1)  # polite pause between requests

print(f"🎉 All {len(t_url)} URLs fetched successfully!")


Retrying missing URLs:   0%|          | 0/147 [00:00<?, ?it/s]

Retrying missing URLs: 100%|██████████| 147/147 [10:09<00:00,  4.14s/it]

🎉 All 143 URLs fetched successfully!


In [8]:
new_input = {input[input['URL'] == url]['URL_ID'].values[0]:resp for url, resp in t_url.items() if resp.status_code == 200}

In [10]:
len(t_url)

143

In [12]:
def get_title_and_text(url, resp):
    soup = BeautifulSoup(resp.content, 'html.parser')

    # Title (first <h1>)
    title_tag = soup.find('h1')
    title = title_tag.get_text(strip=True) if title_tag else ""

    # Get all relevant content containers
    containers = soup.find_all('div', class_='td-post-content tagdiv-type') + soup.find_all(True, class_='wp-block-list')

    # Unwrap unwanted tags globally
    for tag in soup.find_all('div', class_='wp-block-heading'):
        tag.unwrap()
    for tag in soup.find_all('a'):
        tag.unwrap()

    # Extract <p> and <li> text from all containers
    paragraphs = []
    for container in containers:
        paragraphs.extend([
            tag.get_text(strip=True)
            for tag in container.find_all(['p', 'ol', 'ul'])
            if tag.get_text(strip=True)
        ])

    return title, paragraphs

# Usage
# ttl, p = get_title_and_text('Netclan20241022', new_input['Netclan20241022'])
# print(f"Title: {ttl}\nParagraphs: {p}")


In [ ]:
for id in tqdm(new_input.keys(), desc="Processing URLs"):
    ttl, p = get_title_and_text(id, new_input[id])
    if not ttl or not p:
        continue
    text = '{ttl}\n\n{p}'.format(ttl=ttl, p='\n'.join(p))
    
    filepath = fr"C:\Users\nitee\OneDrive\Desktop\VS CODE\BlackCoffer-TestAssginment\Aritcle_txt_files\{id}.txt"
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(text)


Processing URLs: 100%|██████████| 143/143 [00:07<00:00, 19.30it/s]
